YES24 크롤링

In [20]:
# webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

KeyboardInterrupt: 

In [35]:
from selenium import webdriver
browser = webdriver.Chrome()
browser.get('https://google.com')

In [ ]:
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME, 'uU7dJb').text

'대한민국'

In [36]:
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)    

In [ ]:
# 1페이지의 링크 데이터 전부 수집
## 한 개의 베스트 셀러 링크 수집 (1개: 요소)
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

## 여러개 (전체 베스트셀러) 수집
datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # 리스트 데이터 형태로 가져옴

for i in datas:
    print(i.get_attribute('href')) # 리스트 형태로 받은 데이터 중 링크만 print

None
None


In [37]:
# # 3페이지까지의 링크 데이터 전부 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
import time

link_list = []
for i in range (1,4):
    print(f"현재 {i}페이지 수집 중 입니다.")
    url = f"https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24"
    browser.get(url)

    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # 리스트 데이터 형태로 가져옴

    for i in datas:
        link = i.get_attribute('href') # 리스트 형태로 받은 데이터 중 링크만 print
        link_list.append(link)

    time.sleep(3)


현재 1페이지 수집 중 입니다.
현재 2페이지 수집 중 입니다.
현재 3페이지 수집 중 입니다.


In [ ]:
len(link_list)
print(link_list)

['https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/128133897', 'https://www.yes24.com/Product/Goods/124225606', 'https://www.yes24.com/Product/Goods/123930880', 'https://www.yes24.com/Product/Goods/127094902', 'https://www.yes24.com/Product/Goods/123930891', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/126511412', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/126845471', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/126702038', 'https://www.yes24.com/Product/Goods/126516935', 'https://www.yes24.com/Product/Goods/126590468', 'https://www.yes24.com/Product/Goods/8759796', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.co

In [46]:
# for link in link_list:
#     # 상세페이지로 이동
#     browser.get(link)

browser.get(link_list[36])


# title = browser.find_element(By.CLASS_NAME, 'gd_name').text
# author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
# publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
# publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
# rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
if rating:
    try:
        rating = float(rating.replace(',', ''))  # 콤마 제거 및 float로 변환
    except ValueError:
        rating = 0.0  # 기본값 설정
else:
    rating = 0.0  # 기본값 설정
# review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
# sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
# price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
# ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[1][:-1]
# ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]
print(rating)


0.0


In [2]:
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import re
from datetime import datetime

# WebDriver 인스턴스를 생성 (ChromeDriver 경로를 지정하세요)
browser = webdriver.Chrome()

link_list = []
for i in range(1, 4):
    print(f"현재 {i}페이지 수집 중 입니다.")
    url = f"https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24"
    browser.get(url)
    time.sleep(3)  # 페이지 로드 대기

    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')  # 리스트 데이터 형태로 가져옴
    for data in datas:
        link = data.get_attribute('href')  # 리스트 형태로 받은 데이터 중 링크만 추출
        link_list.append(link)

    time.sleep(3)  # 페이지 수집 후 대기

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='oz-password',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)
        time.sleep(3)  # 페이지 로드 대기

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # publishing: 날짜형태
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year, month, day = match.groups()
            date_obj = datetime(int(year), int(month), int(day))
            publishing = date_obj.strftime('%Y-%m-%d')
        else:
            publishing = '2024-01-01'
        
        try:
            rating_element = browser.find_element(By.CLASS_NAME, 'yes_b')
            rating = rating_element.text
        except:
            # 'yes_b' 클래스가 없는 경우, 'gd_reviewCount' 클래스를 사용하거나 기본값 0
            rating = '0'
        
        try:
            rating_area = browser.find_element(By.CLASS_NAME, 'gd_ratingArea')
            try:
                rating_element = rating_area.find_element(By.CLASS_NAME, 'yes_b')
                rating = rating_element.text
                rating = float(rating.replace(',', '')) if rating else 0.0
            except NoSuchElementException:
                rating = 0.0
        except NoSuchElementException:
            rating = 0.0
        
        try:
            review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
            review = int(review.replace(',', ''))
        except:
            review = 0

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(',', ''))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(',', ''))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(" | ")
        try:
            ranking_part = parts[0]
            ranking = ''.join(filter(str.isdigit, ranking_part))  # 숫자만 추출
        except:
            ranking = 0
        
        try:
            ranking_weeks_part = parts[1]
            ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))  # 숫자만 추출
        except:
            ranking_weeks = 0

        sql = """INSERT INTO Books (
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()
        time.sleep(3)  # 다음 링크로 넘어가기 전 대기

# WebDriver 종료
browser.quit()

# 데이터베이스 연결 종료
conn.close()


현재 1페이지 수집 중 입니다.


KeyboardInterrupt: 